# Perspectives on COVID-19: USA and UK

I am curious about COVID-19, and this notebook is my effort to find context and perspective from responsible, public data.

All data used is from:

- [The *New York Times*](https://github.com/nytimes/covid-19-data)
- [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm)
- [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)
- [South Carolina Department of Health and Environmental Control (DHEC)](https://www.scdhec.gov/vital-records/parentage/sc-vital-records-data-and-statistics)
- [The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)

## Configuring Libraries for the Almond Kernel

First, we'll make a bintray repository with libraries available to your almond kernel.

In [ ]:
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")

interp.repositories() ++= Seq(myBT)



Next, we bring in specific libraries from the new repository using almond's `$ivy` magic:

In [ ]:
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._

// if you want to have the plots available without an internet connection:
init(offline=true)

// restrict the output height to avoid scrolling in output cells
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

## Imports

From this point on, your notebook consists of completely generic Scala, with the CITE Libraries available to use.

In [ ]:
import almond.display.UpdatableDisplay
import almond.interpreter.api.DisplayData.ContentType
import almond.interpreter.api.{DisplayData, OutputHandler}

import java.io.File
import java.io.PrintWriter

import scala.io.Source

import java.text.SimpleDateFormat
import java.util.Date


## Useful Functions

Save a string to a names file:

In [ ]:
def saveString(s:String, filePath:String = "", fileName:String = "temp.txt"):Unit = {
		 val writer = new PrintWriter(new File(s"${filePath}${fileName}"))
         writer.write(s)
         writer.close()
	}

Like `.split`, but preserving the character we split on:

In [ ]:
def splitWithSplitter(text: String, puncs: String): Vector[String] = {
	//val regexWithSplitter = s"((?<=${puncs})|(?=${puncs}))"
    val regexWithSplitter = s"((?<=${puncs}))"
	text.split(regexWithSplitter).toVector.filter(_.size > 0)
}

Pretty Print Things:

In [ ]:
def showMe(v:Any):Unit = {
  v match {
    case _:Vector[Any] => println(s"""\n----\n${v.asInstanceOf[Vector[Any]].mkString("\n")}\n----\n""")
    case _:Iterable[Any] => println(s"""\n----\n${v.asInstanceOf[Iterable[Any]].mkString("\n")}\n----\n""")
    case _ => println(s"\n-----\n${v}\n----\n")
  }
}

## Load Some Data

Load up-to-date data from the NY Times. Source: <https://github.com/nytimes/covid-19-data>.

In [ ]:
val dataLines: Vector[String] = {
    scala.io.Source.fromURL("https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-states.csv").mkString.split("\n").toVector
}

// quick test
val badLines = dataLines.filter( l => {
    l.split(",").size != 5
})

assert ( badLines.size == 0 )

## Comparative Data

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

Source for South Carolina Data: [“Stats of the State of South Carolina”, CDC](https://www.cdc.gov/nchs/pressroom/states/southcarolina/southcarolina.htm)

### USA

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

In [ ]:
val usa_2017_Total_Deaths: Int = 2813502

// Chronic lower respiratory diseases
val usa_2017_Total_Deaths_CLRD: Int = 160201

// Influenza and Pneumonia
val usa_2017_Total_Deaths_IPn: Int = 55672

// All respiratory-related deaths
val usa_2017_Total_Deaths_Resp: Int = usa_2017_Total_Deaths_CLRD + usa_2017_Total_Deaths_IPn

// Suicide
val usa_2017_Suicide_Total: Int = 47173

// Heart Disease
val usa_2017_Heart_Total: Int = 647457

// Accidents
val usa_2017_Accident_Total: Int = 169936

// Cancer
val usa_2017_Cancer_Total: Int = 599108



### England & Wales

Source for Data: [The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales). Unfortunately, this data is published in elaborately-formatted Excel (`.xsls`) files, so I have extracted some plain values below. The original files are downloadable from the link above.

In [ ]:
val uk_weeks = Vector("3-Jan-20","10-Jan-20","17-Jan-20","24-Jan-20","31-Jan-20","7-Feb-20","14-Feb-20","21-Feb-20","28-Feb-20","6-Mar-20","13-Mar-20","20-Mar-20","27-Mar-20","3-Apr-20","10-Apr-20","17-Apr-20")
val uk_Death = Vector(12254,14058,12990,11856,11612,10986,10944,10841,10816,10895,11019,10645,11141,16387,18516,22351)
val uk_Deaths_5yr_Ave = Vector(12175,13822,13216,12760,12206,11925,11627,11548,11183,11498,11205,10573,10130,10305,10520,10497)
val uk_2018_Resp_Deaths = Vector(2361,3075,2829,2672,2479,2197,2081,2128,1900,2302,2271,1946,1657,1761,1971,1641)
val uk_2020_Resp_Deaths = Vector(2141, 2477, 2189, 1893, 1746, 1572, 1602, 1619, 1546, 1584, 1508, 1530, 1513, 1951, 1757,1776)
val uk_Covid_Deaths = Vector(0,0,0,0,0,0,0,0,0,0,5,103,539,3475,6213,8758)

// A little testing
assert(
    (uk_weeks.size == uk_Death.size) &&
    (uk_weeks.size == uk_Deaths_5yr_Ave.size) &&
    (uk_weeks.size == uk_2018_Resp_Deaths.size) &&
    (uk_weeks.size == uk_2020_Resp_Deaths.size) &&
    (uk_weeks.size == uk_Covid_Deaths.size)
)

## Make Data Structures

For State data-points from the NY Times:

In [ ]:
case class StateDatum( date: String, state: String, cases: Int, deaths: Int)

For Aggregate data (just a Vector of the above):

In [ ]:
case class RunningTally( days: Vector[StateDatum] )

A daily snapshot (for the USA or a single state)

In [ ]:
case class DailySnapshot( date: String, newCases: Int, newDeaths: Int, totalCases: Int, totalDeaths: Int)

## Load Data

In [ ]:
val vst: Vector[StateDatum] = dataLines.tail.map( dl => {
    val fields: Vector[String] = dl.split(",").toVector
    val date: String = fields(0)
    val state: String = fields(1)
    val cases: Int = fields(3).toInt
    val deaths: Int = fields(4).toInt
    StateDatum(date, state, cases, deaths)
})

val rt = RunningTally(vst)


## Data Functions

Taking, by default, our running-tally data (`rt`) and an optional `Option[String]` that can specify a state (defaults to `None`), return a `Vector[DailySnapshot]`, which will include new cases, new deaths, total cases, and total deaths.

In [ ]:
def totalNewDaily(data: Vector[StateDatum] = rt.days, state: Option[String] = None): Vector[DailySnapshot] = {
    val sortedData: Vector[StateDatum] = {
        val filtered: Vector[StateDatum] = {
            state match {
                case Some(s) => data.filter(_.state == s)
                case None => {
                    val groupedByDay: Vector[(String, Vector[StateDatum])] = {
                        data.groupBy(_.date).toVector
                    }
                    val merged: Vector[StateDatum] = groupedByDay.map( gbd => {
                        val vec = gbd._2
                        val vecDate: String = vec.head.date
                        val vecState: String = "USA"
                        val vecCases: Int = vec.map(_.cases).sum
                        val vecDeaths: Int = vec.map(_.deaths).sum
                        StateDatum(vecDate, vecState, vecCases, vecDeaths)
                    })
                    merged
                }
            }
        }
        val sorted: Vector[StateDatum] = filtered.sortBy(_.date)
        sorted
    }
    // We don't want a running, cumulative tally, but new cases/deaths each day
    sortedData.zipWithIndex.map( sd => {
        val d: StateDatum = sd._1
        val i: Int = sd._2
        val newCases: Int = {
            if (i == 0) d.cases
            else {
                val totalToday: Int = d.cases
                val totalPrev: Int = {
                    sortedData(i-1).cases
                }
                totalToday - totalPrev
            }
        }
        val newDeaths: Int = {
            if (i == 0) d.deaths
            else {
                val totalToday: Int = d.deaths
                val totalPrev: Int = {
                    sortedData(i-1).deaths
                }
                totalToday - totalPrev
            }
        }
        DailySnapshot(d.date, newCases, newDeaths, d.cases, d.deaths)
    })
    
}

## The Curve: Total Cases, USA

Just 2020 COVID-19 Cases, cumulative, as reported by the New York Times (https://github.com/nytimes/covid-19-data).

**This is the scary graph!** The cases keep piling up!

In [ ]:
val curve_total_cases: Vector[Int] = totalNewDaily(state = None).map(_.totalCases)

val curve_total_cases_trace = Scatter(
  (1 to curve_total_cases.size),
  curve_total_cases,
  name = "2020 Covid Total Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)


val data = Seq(curve_total_cases_trace)

val layout = Layout("Total USA COVID Cases")

plot(data, layout)

## The Curve: Total Deaths, USA

Just 2020 COVID-19 Deaths, cumulative as reported by the New York Times (https://github.com/nytimes/covid-19-data).

**This is another scary graph!** The deaths keep piling up!

In [ ]:
val curve_total_deaths: Vector[Int] = totalNewDaily(state = None).map(_.totalDeaths)

val curve_total_deaths_trace = Scatter(
  (1 to curve_total_deaths.size),
  curve_total_deaths,
  name = "2020 Covid Death-Toll",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_deaths_trace)

val layout = Layout("Total US Deaths")

plot(data, layout)

## The Curve: Total Cases & Deaths, USA

Just 2020 COVID-19 Deaths, cumulative as reported by the New York Times (https://github.com/nytimes/covid-19-data).

Juxatposing the above two graphs.

In [ ]:
val data = Seq(curve_total_deaths_trace, curve_total_cases_trace)

val layout = Layout("Total USA COVID Cases and Deaths")

plot(data, layout)

## Case-Fatality-Rate: USA

A rolling calculation of deaths/cases. Overlaid with total cases and total deaths. 

So the Case Fatality Rate is visible on a scale with the other two, it is multiplied by 10,000,000. So `800k` on the gray line = `0.08%`.

Hover over a point on the line to see the actual percent.

In [ ]:
val cfr: Vector[Double] = curve_total_deaths.zip(curve_total_cases).map( c => {
    c._1.toDouble / c._2.toDouble
})

val cfr50: Vector[Double] = curve_total_deaths.zip(curve_total_cases).map( c => {
    c._1.toDouble / (c._2.toDouble * 20)
})

val cfr85: Vector[Double] = curve_total_deaths.zip(curve_total_cases).map( c => {
    c._1.toDouble / (c._2.toDouble * 50)
})

val case_fatality_rate = Scatter(
  (1 to cfr.size),
  cfr.map(_ * 10000000),
  text = cfr.zipWithIndex.map(z => {
      val d = z._1
      val i = z._2
      val cases = curve_total_cases(i)
      val deaths = curve_total_deaths(i)
      val percent = (d * 100)
      s"${deaths} deaths / ${cases} cases -> " + f"$d%1.4f" + " = " + f"$percent%1.2f" + "%"
  }),
  name = "Case Fatality Rate",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val case_fatality_rate_50 = Scatter(
  (1 to cfr50.size),
  cfr50.map(_ * 10000000),
  text = cfr50.zipWithIndex.map(z => {
      val d = z._1
      val i = z._2
      val cases = curve_total_cases(i) * 10
      val deaths = curve_total_deaths(i)
      val percent = (d * 100)
      s"${deaths} deaths / ${cases} cases -> " + f"$d%1.4f" + " = " + f"$percent%1.2f" + "%"
  }),
  name = "Case Fatality Rate (10x cases)",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val case_fatality_rate_85 = Scatter(
  (1 to cfr85.size),
  cfr85.map(_ * 10000000),
  text = cfr85.zipWithIndex.map(z => {
      val d = z._1
      val i = z._2
      val cases = curve_total_cases(i) * 20
      val deaths = curve_total_deaths(i)
      val percent = ((d) * 100)
      s"${deaths} deaths / ${cases} cases -> " + f"$d%1.4f" + " = " + f"$percent%1.2f" + "%"
  }),
  name = "Case Fatality Rate (20x cases)",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(case_fatality_rate, case_fatality_rate_50, case_fatality_rate_85)

val layout = Layout("2020 Covid Case Fatality Rate")

plot(data, layout)

## New Cases vs. Total Cases

This juxtaposition is informative.

In [ ]:

val new_cases_usa: Vector[Int] = {
    totalNewDaily().map( z => z.newCases)
}

val curve_new_cases_trace = Scatter(
  (1 to new_cases_usa.size),
  new_cases_usa,
  name = "2020 Covid New Cases, Daily",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_cases_trace, curve_new_cases_trace)

val layout = Layout(s"New Cases vs. Total Cases: USA")

plot(data, layout)

## US: New Cases

Just new cases, for detail. Also, averaged over N-days (set the value of `n` below).

In [ ]:

val nDays = 7

val new_cases_usa: Vector[Int] = {
    totalNewDaily().map( z => z.newCases)
}

val curve_new_cases_nave_trace = Scatter(
  (1 to new_cases_usa.size),
  new_cases_usa.sliding(nDays,1).toVector.map( n => {
      n.sum / n.size
  }),
  name = s"2020 Covid New Cases, Daily ${nDays} ave.",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(230, 230, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val curve_new_cases_trace = Scatter(
  (1 to new_cases_usa.size),
  new_cases_usa,
  name = "2020 Covid New Cases, Daily",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 0, 0.3),
    line = Line(
      color = Color.RGBA(230, 230, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_new_cases_nave_trace, curve_new_cases_trace)

val layout = Layout(s"New Cases: USA")

plot(data, layout)

## US: New Deaths

Just new deaths, for detail.

In [ ]:

val nDays4Deaths: Int = 7

val new_deaths_usa: Vector[Int] = {
    totalNewDaily().map( z => z.newDeaths)
}

val curve_new_deaths_nave_trace = Scatter(
  (1 to new_deaths_usa.size),
  new_deaths_usa.sliding(nDays,1).toVector.map( n => {
      n.sum / n.size
  }),
  name = s"2020 Covid New Deaths, Daily ${nDays4Deaths} ave.",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val curve_new_deaths_trace = Scatter(
  (1 to new_deaths_usa.size),
  new_deaths_usa,
  name = "2020 Covid New Deaths, Daily",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.3),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)



val data = Seq(curve_new_deaths_trace, curve_new_deaths_nave_trace)

val layout = Layout(s"New Deaths: USA")

plot(data, layout)

## US: New Cases vs. New Deaths

In [ ]:



val data = Seq(curve_new_cases_nave_trace, curve_new_deaths_nave_trace, curve_new_cases_trace, curve_new_deaths_trace)

val layout = Layout(s"New Cases vs. New Deaths: USA")

plot(data, layout)

## “Active” Cases, USA

Simply piling up “cases” is misleading. Patients get over COVID-19, or they die. This graph overlays “total cases” with a graph of cases, with cases a certain number of days in the past removed, and deaths removed.

### Set the Threshold!

Set the value below for the number of days in which a patient either dies or is considered “recovered”.

> By setting this to a smaller number than seems reasonable, you can do a kind of “faux-modelling”, to see, now, what a realistic curve might look like in the future.

In [ ]:
val threshold: Int = 14

In [ ]:
val active_cases_usa: Vector[Int] = {
    totalNewDaily().zipWithIndex.map( z => {
        val tnd: DailySnapshot = z._1
        val i: Int = z._2
        if (i < threshold) tnd.totalCases
        else {
            val recovered: Int = totalNewDaily()(i - threshold).totalCases
            val dead: Int = totalNewDaily()(i - threshold).totalDeaths
            tnd.totalCases - (dead + recovered)
        }
    })
}

val curve_active_cases_trace = Scatter(
  (1 to active_cases_usa.size),
  active_cases_usa,
  name = "2020 Covid Active Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 0, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_active_cases_trace, curve_new_cases_trace, curve_total_cases_trace)

val layout = Layout(s"Active Cases: USA (Minus Recovered and Dead): threshold = ${threshold} days.")

plot(data, layout)

## Recovered Cases, USA



In [ ]:
// totalNewDaily


val recovered_cases_usa: Vector[Int] = {
    totalNewDaily().zipWithIndex.map( z => {
        val tnd: DailySnapshot = z._1
        val i: Int = z._2
        if (i < threshold) 0
        else {
            val recovered: Int = totalNewDaily()(i - threshold).totalCases
            val dead: Int = totalNewDaily()(i - (threshold/2)).totalDeaths
            recovered - dead
        }
     })
}

val curve_recovered_cases_trace = Scatter(
  (1 to recovered_cases_usa.size),
  recovered_cases_usa,
  name = "2020 Covid Recovered Cases",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 0, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_recovered_cases_trace, curve_total_cases_trace)

val layout = Layout(s"Recovered Cases & Total Cases: threshold = ${threshold} days.")

plot(data, layout)

## Compared to the Flu

Throughout a year like 2017, the death-toll for Influenzas and Pneumonias mounts, day to day. Here we chart that death-toll, assuming the annual deaths are evenly distributed across the year. 

But the flu-season is, as defined by the CDC, 6 months, from October through March. So we can also compare daily flu deaths and their mounting toll, over a six-month season.

Source for USA Data: [“Deaths and Mortality”, CDC](https://www.cdc.gov/nchs/fastats/deaths.htm).

In [ ]:
val usa_2017_mounting_flu_toll: Vector[Int] = {
    val daily = usa_2017_Total_Deaths_IPn / 365
    (1 to curve_total_deaths.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val usa_2017_mounting_flu_toll_6: Vector[Int] = {
    val daily = usa_2017_Total_Deaths_IPn / 365 * 2
    (1 to curve_total_deaths.size).toVector.map( i => {
        if (i == 1) daily
        else daily + (daily * (i - 1))
    })
}

val usa_2017_Daily_Deaths_IPn_trace = Scatter(
  (1 to curve_total_deaths.size),
  usa_2017_mounting_flu_toll,
  name = "2017 Influenza Death Toll",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(204, 204, 0, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val usa_2017_Daily_Deaths_IPn_6mo_trace = Scatter(
  (1 to curve_total_deaths.size),
  usa_2017_mounting_flu_toll_6,
  name = "2017 Influenza Death Toll (6 mo. season)",
  mode = ScatterMode(ScatterMode.Lines),
  marker = Marker(
    color = Color.RGBA(0, 204, 204, 0.95),
    line = Line(
      color = Color.RGBA(217, 217, 217, 1.0),
      width = 1.0
    ),
    symbol = Symbol.Circle(),
    size = 3
  )
)

val data = Seq(curve_total_deaths_trace, usa_2017_Daily_Deaths_IPn_trace, usa_2017_Daily_Deaths_IPn_6mo_trace)

val layout = Layout("COVID Death Toll vs. Flu Season, 2017")

plot(data, layout)

## Are More People Dying?

We do not have have good, readily available data for the USA to determine this.

For England and Wales, however, we have week-by-week data for deaths (from all causes), in 2020, through the week of March 27, and an *average* of deaths in each of the first 13 weeks in 2015-2019.

Source for Data:[The Office of National Statistics, UK](https://www.ons.gov.uk/peoplepopulationandcommunity/birthsdeathsandmarriages/deaths/datasets/weeklyprovisionalfiguresondeathsregisteredinenglandandwales)

In [ ]:

val cumuUkDeathsByWeek2020: Vector[Int] = {
    uk_Death.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (uk_Death.take(i+1).sum)
        }
    })
}

val cumuUkDeathsByWeekAve: Vector[Int] = {
    uk_Deaths_5yr_Ave.zipWithIndex.map( t => {
        val d = t._1
        val i = t._2
        if (i == 0 ) {
            d
        }
        else {
            d + (uk_Deaths_5yr_Ave.take(i+1).sum)
        }
    })
}

val trace_uk_death_2020 = Scatter(
  (1 to cumuUkDeathsByWeek2020.size).toSeq,
  cumuUkDeathsByWeek2020.toSeq,
  fill = Fill.ToZeroY, 
  name = "2020"
)

val trace_uk_death_5yr_ave = Scatter(
  (1 to cumuUkDeathsByWeek2020.size).toSeq,
  cumuUkDeathsByWeekAve.toSeq,
  fill = Fill.ToNextY,
  name = "Average (2015-2019)"
)

val data = Seq(trace_uk_death_2020, trace_uk_death_5yr_ave)

val layout = Layout("England and Wales: Weekly Death Toll, all causes")

plot(data, layout)